# Import Data

## Packages

In [ ]:
import pandas as pd
import numpy as np
from plotnine import *
import seaborn as sns

In [18]:
# clean df

In [20]:
# clean + engineered features

# Train Models

## Baseline Linear Regression

This model will be trained on the clean data (without engineered features) to understand what our baseline statistics look like.

### Import Packages

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

### Assign Variable Groups and Create Train/Test Split

In [ ]:
quantitative_vars = [
    # Property characteristics
    "accommodates",
    "bathrooms",
    "bedrooms", 
    "beds",
    
    # Location
    "latitude",
    "longitude",
    
    # Host metrics
    "host_response_rate",
    "host_acceptance_rate",
    "host_listings_count",
    "host_total_listings_count",
    "calculated_host_listings_count",
    "calculated_host_listings_count_entire_homes",
    "calculated_host_listings_count_private_rooms",
    "calculated_host_listings_count_shared_rooms",
    
    # Reviews
    "number_of_reviews",
    "number_of_reviews_ltm",
    "number_of_reviews_l30d",
    "number_of_reviews_ly",
    "reviews_per_month",
    "review_scores_rating",
    "review_scores_accuracy",
    "review_scores_cleanliness",
    "review_scores_checkin",
    "review_scores_communication",
    "review_scores_location",
    "review_scores_value",
    
    # Availability
    "availability_30",
    "availability_60",
    "availability_90",
    "availability_365",
    "availability_eoy",
    "minimum_nights",
    "maximum_nights",
    "minimum_nights_avg_ntm",
    "maximum_nights_avg_ntm",
    
    # Other
    "estimated_occupancy_l365d",
    "estimated_revenue_l365d"
]

categorical_vars = [
    # Core categories
    #"neighbourhood_cleansed", --> Removed to reduce overfitting, increased Rsq substantially
    #"property_type",          --> Removed because too many categories, OHE could not handle
    "room_type",
    
    # Host attributes
    "host_is_superhost",
    "host_has_profile_pic",
    "host_identity_verified",
    
    # Listing attributes
    "instant_bookable",
    "has_availability"#,
    
    # License
    #"license"                 --> Removed because too many categories, OHE could not handle
]

ignore = [
    "id",
    "listing_url",
    "scrape_id",
    "last_scraped",
    "source",
    "name",
    "picture_url",
    "host_id",
    "host_url",
    "host_name",
    "host_since",
    "host_thumbnail_url",
    "host_picture_url",
    "host_verifications",
    "calendar_last_scraped",
    "amenities",  # Need to parse separately
    "log_price",  # This is your target
    "price",  # Original target
    "property_type",
    "license",
    "neighbourhood_cleansed"
]

target = "log_price"

X = df_clean.drop(ignore, axis = 1)
y = df_clean[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Preprocess

In [ ]:
preproc = make_column_transformer(
    (StandardScaler(), quantitative_vars),
    (OneHotEncoder(drop = "first"), categorical_vars)
)

### Define and Fit

In [ ]:
Linear_Pipeline = make_pipeline(
    preproc,
    LinearRegression()
)

Linear_Pipeline.fit(X_train, y_train)

### Predict

In [ ]:
y_pred = Linear_Pipeline.predict(X_test)
y_pred_train = Linear_Pipeline.predict(X_train)

### Results

In [ ]:
print("MSE: ", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAE: ", mean_absolute_error(y_test, y_pred))
print("R Squared (Test): ", r2_score(y_test, y_pred))
print("R Squared (Train): ", r2_score(y_train, y_pred_train))

### Observations and Conclusions

Trained the linear model multiple times for various results. Made the following observations:
- Variables "property_type" and "license" both have too many categories and caused the model to throw errors.
- Removing "neighbourhood_cleansed" reduced overfitting and increased R squared substantially.
- There is a large amount of multicollinearity between variables like "accommodates" and "beds" which move similarily.

## Linear Regression (With Engineered Features)